In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

Installed package MathNet.Numerics.FSharp version 4.15.0

In [ ]:
open MathNet.Numerics.LinearAlgebra
open System.Collections.Generic

In [ ]:
let p = fun x -> 1. + x
// b = 0, t = 0

let u = fun x t -> x ** 3. + t ** 3.
let f = fun x t -> 3. * (t ** 2. - 3. * x ** 2. - 2. * x)
let phi = fun (x: float) -> x ** 3.
let alpha = fun (t: float) -> 0.
let beta = fun (t: float) -> t ** 3. + 1.

let left = 0.
let right = 1.
let T = 0.1

In [ ]:
let n = 5
let h = (right - left) / float n
let x = [ left .. h .. right ]

let m = 5
let tau = T / float m
let t = [ 0. .. tau .. T ]

In [ ]:
let l i k (u: Matrix<float>) = p ((float i + 0.5) / float n) * (u.[i + 1, k] - u.[i, k]) / h ** 2. - p ((float i - 0.5) / float n) * (u.[i, k] - u.[i - 1, k]) / h ** 2.

In [ ]:
let explicitScheme n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    x |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    for k = 1 to m do
        let column = DenseVector.init (m + 1) (fun i ->
            if i = 0 then (-2. * alpha t.[k] / float n + 4. * u.[1, k] - u.[2, k]) / 3.
            elif i = n then beta t.[k]
            else u.[i, k - 1] + tau * (l i (k - 1) u + f x.[i] t.[k - 1])
        )

        u.SetColumn(k, column)

    u

In [ ]:
let weightScheme w n m = 
    let u = DenseMatrix.zero<float> (n + 1) (m + 1)

    // из начальных условий
    x |> List.iteri (fun i x -> u.[i, 0] <- phi x)

    let a = [ for i = 0 to n - 2 do yield (w * p ((float i + 0.5) / float n) / h ** 2.) ] @ [0.]
    let b = [ yield float n; for i = 1 to n - 1 do yield (w * (p ((float i + 0.5) / float n) + p ((float i - 0.5) / float n)) / h ** 2. + 1. / tau) ] @ [-1.]
    let c = [ yield float n; for i = 1 to n - 1 do yield (w * p ((float i + 0.5) / float n) / h ** 2.) ]

    // решение в виде y_i = s_i*y_i+1 + t_i
    let solveSweep (a: float list) (b: float list) (c: float list) (g: float list) = 
        let n = List.length a

        // из нулевого уравнения системы
        let s = List<float>()
        let t = List<float>()
        s.Add(c.[0] / b.[0])
        t.Add(-g.[0] / b.[0])

        // рекурентные формулы
        for i = 1 to n - 1 do
            s.Add(c.[i] / b.[i] - a.[i - 1] * s.[i - 1])
            t.Add((a.[i - 1] * t.[i - 1] - g.[i]) / b.[i] - a.[i - 1] * s.[i - 1])

        let y = List<float>() 
        y.Add((a.[n - 1] * t.[n - 1] - g.[n]) / (b.[n] - a.[n - 1] * s.[n - 1]))

        for i = n - 1 downto 0 do
            y.Add(s.[i] * y.[y.Count - 1] + t.[i])

        // обратная прогонка
        for i = 0 to y.Count / 2 - 1 do
            let temp = y.[i]
            y.[i] <- y.[y.Count - 1 - i]   
            y.[y.Count - 1 - i] <- temp

        y

    for k = 1 to m do
        // Gik
        let g = DenseVector.init (n + 1) (fun i ->
            if i = 0 then alpha t.[k]
            elif i = n then beta t.[k]
            else -1. / tau * u.[i, k - 1] - (1. - w) * l i (k - 1) u - f x.[i] t.[k] - (1. - w) * tau)
        ) 
        
        let s = solveSweep a b c (Vector.toList g)

        for i = 0 to n do
            u.[i, k] <- s.[i]

    u

In [ ]:
printfn "Точное решение в узлах решетки"
DenseMatrix.init (n + 1) (m + 1) (fun i k -> x.[i] ** 3. + t.[k] ** 3.)
|> Matrix.toRowSeq
|> Seq.mapi (fun i row -> {| X = x.[i]; T0 = row.[0]; T02 = row.[1]; T04 = row.[2]; T06 = row.[3]; T08 = row.[4]; T1 = row.[5] |})

Точное решение в узлах решетки

index,T0,T02,T04,T06,T08,T1,X
0,0,8.000000000000001E-06,6.400000000000001E-05,0.00021599999999999996,0.0005120000000000001,0.0010000000000000002,0
1,0.008000000000000002,0.008008000000000001,0.008064000000000002,0.008216000000000001,0.008512000000000002,0.009000000000000003,0.2
2,0.06400000000000002,0.06400800000000001,0.06406400000000001,0.06421600000000001,0.06451200000000001,0.06500000000000002,0.4
3,0.2160000000000001,0.21600800000000012,0.21606400000000012,0.2162160000000001,0.21651200000000012,0.2170000000000001,0.6000000000000001
4,0.5120000000000001,0.5120080000000001,0.5120640000000001,0.5122160000000001,0.5125120000000001,0.5130000000000001,0.8
5,1,1.000008,1.000064,1.000216,1.000512,1.001,1


In [ ]:
printfn "Решение явной разностной схемой"
explicitScheme n m
|> Matrix.toRowSeq
|> Seq.mapi (fun i row -> {| X = x.[i]; T0 = row.[0]; T02 = row.[1]; T04 = row.[2]; T06 = row.[3]; T08 = row.[4]; T1 = row.[5] |})

Решение явной разностной схемой

index,T0,T02,T04,T06,T08,T1,X
0,0,0,0,0,0,0,0
1,0.008000000000000002,0.008800000000000002,0.009184000000000013,0.00971480000000001,0.010125139999999835,0.010479759250000251,0.2
2,0.06400000000000002,0.06480000000000004,0.06562400000000003,0.06623399999999977,0.06664736500000035,0.06772426199999904,0.4
3,0.2160000000000001,0.21680000000000005,0.21762399999999973,0.21788046000000036,0.21895715599999907,0.21867673690000128,0.6000000000000001
4,0.5120000000000001,0.5127999999999998,0.5128716000000002,0.5136399199999995,0.5135076550000006,0.5149778585999987,0.8
5,1,1.000008,1.000064,1.000216,1.000512,1.001,1


In [ ]:
printfn "Решение схемой с весом 0.5"
weightScheme 0.5 n m
|> Matrix.toRowSeq
|> Seq.mapi (fun i row -> {| X = x.[i]; T0 = row.[0]; T02 = row.[1]; T04 = row.[2]; T06 = row.[3]; T08 = row.[4]; T1 = row.[5] |})

Решение схемой с весом 0.5

index,T0,T02,T04,T06,T08,T1,X
0,0,2154279300371.9321,2.2293460113954272E+17,2.307009970238644E+22,2.387379516302018E+27,2.4705489045931054E+32,0
1,0.008000000000000002,2154279300371.9321,2.2293460113954272E+17,2.307009970238644E+22,2.387379516302018E+27,2.4705489045931054E+32,0.2
2,0.06400000000000002,-159024077537.12106,-16456468713613190,-1.7029764425058765E+21,-1.7623032097962048E+26,-1.823696749843496E+31,0.4
3,0.2160000000000001,-721664645.1631229,-74683341055687.69,-7.728509239371981E+18,-7.997748118842758E+23,-8.27636650113527E+28,0.6000000000000001
4,0.5120000000000001,174674.31994201237,18096484630.2342,1872691395546492,1.9379305402918303E+20,2.0054424279017002E+25,0.8
5,1,1.000008,1.000064,1.000216,1.000512,1.001,1
